## Обязательно запустить код в ячейке ниже. Это генератор датасета.

In [ ]:
! pip install Faker
import pandas as pd
import numpy as np
import random
from faker import Faker
import warnings
warnings.filterwarnings('ignore')

random.seed(42)
np.random.seed(42)

Faker.seed(42)
fake = Faker()

def generate_random_data():
    passenger_id = random.randint(1, 1000)
    survived = random.choice([0, 1])
    pclass = random.choice([1,2,3,'I','II','III'])
    name = fake.name()
    sex = random.choice(['male', 'female', 'Female', 'Male', 'FEMALE', 'MALE'])
    age = random.uniform(-1, 80)
    sibsp = random.randint(0, 5)
    parch = random.randint(0, 5)
    ticket = fake.bban()
    fare = round(random.uniform(0, 500), 2)
    cabin = fake.building_number() if random.random() > 0.8 else np.NaN
    embarked = random.choice(['S', 'C', 'Q'])

    return {
        'PassengerId': passenger_id,
        'Survived': survived,
        'Pclass': pclass,
        'Name': name,
        'Sex': sex,
        'Age': age,
        'SibSp': sibsp,
        'Parch': parch,
        'Ticket': ticket,
        'Fare': fare,
        'Cabin': cabin,
        'Embarked': embarked
    }

def add_missing_values(df, columns, missing_count):
    for col in columns:
        indices = df.sample(missing_count).index
        df.loc[indices, col] = np.nan
    return df

def add_noise_to_age_column(df, noise_percentage):
    rows_with_noise = df.sample(frac=noise_percentage)
    for index, row in rows_with_noise.iterrows():
        age_value = row['Age']
        suffixes = ["y.o.", "years", "y"]
        suffix = random.choice(suffixes)
        df.at[index, 'Age'] = f"{age_value:.0f}{suffix}"
    return df

def add_duplicates(df, full_dupes=5, partial_dupes=5):
    full_duplicates = df.sample(full_dupes, replace=True)
    partial_duplicates = df.sample(partial_dupes, replace=True).copy()
    partial_duplicates['PassengerId'] = [random.randint(1001, 2000) for _ in range(partial_dupes)]
    df = pd.concat([df, full_duplicates, partial_duplicates], ignore_index=True)
    return df

data = [generate_random_data() for _ in range(100)]
df = pd.DataFrame(data)
df = add_noise_to_age_column(df, 0.2)
missing_age_count = int(len(df) * 0.15)
age_indices = df.sample(missing_age_count).index
df.loc[age_indices, 'Age'] = np.nan
columns_to_modify = ['Fare', 'Embarked']
df = add_missing_values(df, columns_to_modify, 8)
df = add_duplicates(df, full_dupes=2, partial_dupes=2)


## Почистить датасет во всех колонках от "малых" проблем.

    1) отсутсвующие значения
    2) неподходящие типы данных у колонок
    3) дубликаты (полные и частичные)
    4) проблемы с категориальными переменными
    5) проблемы значений (грязь в данных)

In [ ]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,655,0,1,Allison Hill,MALE,21y,1,1,HEXD81960013389083,368.24,NaN,Q
1,90,1,1,Meredith Barnes,male,6.589314,1,4,OSIZ02654235116155,301.01,NaN,Q
2,666,1,2,Renee Blair,Male,46.73052,0,1,RDMC84959310341316,349.07,NaN,S
3,221,1,1,Mark Diaz,male,29.774111,2,2,LLGV41928327648350,301.86,05641,Q
4,471,0,I,Tommy Walter,male,44y,5,4,KGUP67242388496965,442.73,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
99,473,0,III,Carolyn Fuller,female,NaN,5,2,TYME30913075626368,254.44,NaN,C
100,664,0,II,Alec Hickman,MALE,12.862231,5,1,ZJHB39117182278248,81.70,NaN,Q
101,626,0,I,Perry Saunders,Male,70.77709,4,1,VEUD04709521456232,359.17,NaN,C
102,1306,1,1,Richard Camacho,female,NaN,3,4,QJOU00766177115921,473.03,NaN,Q
